In [2]:
from skt.ye import get_spark
from datetime import datetime, timedelta
import subprocess
from typing import Union

ModuleNotFoundError: No module named 'skt'

In [ ]:
def run_cmd(args: Union[list, str], shell=True, timeout=5):
    if shell:
        if isinstance(args, str):
            print('Running system command: {0}'.format(args))
            proc = subprocess.Popen(args, shell=shell,stdout=subprocess.PIPE)
        elif isinstance(args, list):
            print('Running system command: {0}'.format(' '.join(args)))
            proc = subprocess.Popen(' '.join(args), shell=shell,stdout=subprocess.PIPE)
        else:
            raise TypeError('input should be [List, str]')
    else:
        if isinstance(args, list):
            proc = subprocess.Popen(args, stdout=subprocess.PIPE)
        else:
            raise TypeError
    try:
        s_output, s_err = proc.communicate(timeout=timeout)
    
    except subprocess.TimeoutExpired:    
        proc.kill()
        s_output, s_err = proc.communicate()
    
    return_code =  proc.returncode    

    return s_output.decode("utf-8").strip() , s_err, return_code

In [ ]:
execution_dt = datetime.strptime(current_dt, '%Y%m%d')
execution_date_ago = (execution_dt - timedelta(days=1)).strftime('%Y/%m/%d')
execution_dt = execution_dt.strftime('%Y/%m/%d')
print(f'excution_dt: {execution_dt}')
print(f'execution_dt_ago: {execution_date_ago}')

In [ ]:
hadoop_ip = 'hdfs://172.27.1.237:8020//'

In [ ]:
today = datetime.today()
yesterday = (today - timedelta(days=1)).strftime('%Y%m%d')

In [ ]:
spark = get_spark()

In [ ]:
remote_meta_path = f"{hadoop_ip}/data/music/transform/flo/track_meta/{yesterday}/*"
output_format = (today - timedelta(days=1)).strftime('%Y/%m/%d')
local_meta_path = f"/data/adot/temp/jina/flo/meta/{output_format}"


In [ ]:
jvm = spark._jvm
jsc = spark._jsc
fs = jvm.org.apache.hadoop.fs.FileSystem.get(jsc.hadoopConfiguration())
if fs.exists(jvm.org.apache.hadoop.fs.Path(local_meta_path)):
    print(f"{local_meta_path} exists")
else:
    print(f"{local_meta_path} does not exists")
    
    cmd_query = "hdfs dfs -mkdir -p " + local_meta_path + ";echo $?"
    s_output, stderr, return_code = run_cmd(cmd_query)
    if return_code != 0:
        print(stderr, return_code)
        raise Exception(f'{cmd_query} fail')
        
    cmd_query = "hadoop distcp -pb "+remote_meta_path+ " " + local_meta_path + ";echo $?"
    s_output, stderr, return_code = run_cmd(cmd_query)
    if return_code != 0:
        print(stderr, return_code)
        raise Exception(f'{cmd_query} fail')